In [1]:
import pandas as pd
import geojson
import json
from shapely.geometry import shape, MultiLineString
import math

In [2]:
hidro = pd.read_csv("instalaciones-hidrocarburos-ductos-res-319-93.csv")

In [3]:
hidro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134112 entries, 0 to 134111
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   du                        134112 non-null  object 
 1   trdu                      134039 non-null  object 
 2   empresa_informante        133900 non-null  object 
 3   tipo                      134104 non-null  object 
 4   tipo_tramo                133985 non-null  object 
 5   diametro                  134112 non-null  float64
 6   prof                      134112 non-null  float64
 7   espesor                   134112 non-null  float64
 8   material                  134112 non-null  object 
 9   revestim                  134091 non-null  object 
 10  geojson                   134112 non-null  object 
 11  alta_planos_base          134112 non-null  object 
 12  modificacion_planos_base  69231 non-null   object 
dtypes: float64(3), object(10)
memory usage: 13.3

In [4]:
# Borro las filas con valores Nan en las columnas empresa_informante y tipo. Estas filas no aportar información a
# mis objetivos

hidro.dropna(subset=["empresa_informante","tipo"], inplace=True)

In [5]:
hidro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133892 entries, 123 to 134111
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   du                        133892 non-null  object 
 1   trdu                      133819 non-null  object 
 2   empresa_informante        133892 non-null  object 
 3   tipo                      133892 non-null  object 
 4   tipo_tramo                133773 non-null  object 
 5   diametro                  133892 non-null  float64
 6   prof                      133892 non-null  float64
 7   espesor                   133892 non-null  float64
 8   material                  133892 non-null  object 
 9   revestim                  133872 non-null  object 
 10  geojson                   133892 non-null  object 
 11  alta_planos_base          133892 non-null  object 
 12  modificacion_planos_base  69039 non-null   object 
dtypes: float64(3), object(10)
memory usage: 14.3+ M

In [6]:
hidro.drop(["alta_planos_base", "modificacion_planos_base"], axis=1,inplace=True)

In [7]:
hidro["du"] = hidro["du"].str.upper()
hidro["trdu"] = hidro["trdu"].str.upper()
hidro["material"] = hidro["material"].str.upper()
hidro["revestim"] = hidro["revestim"].str.upper()

In [8]:
hidro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133892 entries, 123 to 134111
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   du                  133892 non-null  object 
 1   trdu                133819 non-null  object 
 2   empresa_informante  133892 non-null  object 
 3   tipo                133892 non-null  object 
 4   tipo_tramo          133773 non-null  object 
 5   diametro            133892 non-null  float64
 6   prof                133892 non-null  float64
 7   espesor             133892 non-null  float64
 8   material            133892 non-null  object 
 9   revestim            133872 non-null  object 
 10  geojson             133892 non-null  object 
dtypes: float64(3), object(8)
memory usage: 12.3+ MB


In [9]:
# Puedo suponer que los registros iguales son de información duplicada.

hidro.drop_duplicates(inplace=True)

In [10]:
# Creo una función para pasar la columna geojson a un objeto geojson

def convertir_a_objeto_geojson(cadena_geojson):
    return geojson.loads(cadena_geojson)


# Convierto la columna geojson a un objeto geojson

hidro['objeto_geojson'] = hidro['geojson'].apply(convertir_a_objeto_geojson)


# Creo una función para calcular la longitud del tramo

def calcular_longitud_multilinestring(objeto_geojson):
    # Convertir el objeto GeoJSON a un objeto Shapely
    shapely_geojson = shape(objeto_geojson)
    if isinstance(shapely_geojson, MultiLineString):
        # Calcular la longitud de las líneas en el MultiLineString
        longitud_total = sum(line.length for line in shapely_geojson.geoms)
        return longitud_total
    else:
        # En caso de que no sea MultiLineString, devolver 0 o algún valor adecuado
        return 0
    
# Calculo sobre la columna objeto_geojson la longitud del tramo

hidro["longitud"] = hidro['objeto_geojson'].apply(calcular_longitud_multilinestring)

# Lo paso a kilómetros

hidro['longitud'] = hidro['longitud']*100

In [11]:
# Completo la columna tipo_tramo según corresponda en función de la columna profundidad

hidro.loc[(hidro["prof"] == 0), "tipo_tramo"] = "AEREO"
hidro.loc[(hidro["prof"] != 0), "tipo_tramo"] = "ENTERRADO"
        

In [12]:
# Completo la columna trdu

hidro.loc[hidro["trdu"].isnull(), "trdu"] = "Sin dato"

In [13]:
hidro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132980 entries, 123 to 134111
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   du                  132980 non-null  object 
 1   trdu                132980 non-null  object 
 2   empresa_informante  132980 non-null  object 
 3   tipo                132980 non-null  object 
 4   tipo_tramo          132980 non-null  object 
 5   diametro            132980 non-null  float64
 6   prof                132980 non-null  float64
 7   espesor             132980 non-null  float64
 8   material            132980 non-null  object 
 9   revestim            132960 non-null  object 
 10  geojson             132980 non-null  object 
 11  objeto_geojson      132980 non-null  object 
 12  longitud            132980 non-null  float64
dtypes: float64(4), object(9)
memory usage: 14.2+ MB


In [14]:
hidro.drop(["espesor", "objeto_geojson"],axis=1, inplace=True)

In [15]:
hidro.head()

,du,trdu,empresa_informante,tipo,tipo_tramo,diametro,prof,material,revestim,geojson,longitud
123,ALTO LAS HORMIGAS,ALTO LAS HORMIGAS-LAS BASES,COMPAÑÍA GENERAL DE COMBUSTIBLES S.A.,GASODUCTO,ENTERRADO,6.000,1.2,ACERO,POLIURETANO EXTRUIDO TRICAPA,"{""type"":""MultiLineString"",""coordinates"":[[[-67...",17.765197
124,DEL T-25,DEL T-25,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO Y CHAPA,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.117905
125,DEL T-47,DEL T-47,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO Y CHAPA,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.181930
126,DEL TR-1014,DEL TR-1014,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO Y CHAPA,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.708452
127,DEL T-36,DEL T-36,PETROLERA EL TREBOL S.A.,OLEODUCTO,AEREO,2.875,0.0,ACERO,POLIURETANO Y CHAPA,"{""type"":""MultiLineString"",""coordinates"":[[[-69...",0.344256


In [16]:
hidro.index.name = "ID"

In [17]:
hidro.to_csv(path_or_buf="hidrocarburos_con_long.csv", index="ID")